In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Bogotá, Colombia",
                    }
                },
                "required": ["location"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
    ,
    {
        "type": "function",
        "function": {
            "name": "get_food_items",
            "description": "Get available food items at a specific location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location_id": {
                        "type": "integer",
                        "description": "ID of the location to check food items"
                    }
                },
                "required": ["location_id"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

stream = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What's the weather like in Paris today and what food items are available?"}],
    tools=tools,
    stream=True,
)


final_tool_calls = {}

for chunk in stream:
    for tool_call in chunk.choices[0].delta.tool_calls or []:
        index = tool_call.index

        if index not in final_tool_calls:
            final_tool_calls[index] = tool_call

        final_tool_calls[index].function.arguments += tool_call.function.arguments
print(final_tool_calls)

{0: ChoiceDeltaToolCall(index=0, id='call_yI9rcOVsRsskyZbZTHgt5ala', function=ChoiceDeltaToolCallFunction(arguments='{"location": "Paris, France"}', name='get_weather'), type='function'), 1: ChoiceDeltaToolCall(index=1, id='call_D9vKkU7OozO55eqcff0R3Lxu', function=ChoiceDeltaToolCallFunction(arguments='{"location_id": 1}', name='get_food_items'), type='function')}


In [2]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Paris today and what food items are available?",
    }
]

for index, tool_call in final_tool_calls.items():
    # Append the assistant message that originally called the function
    messages.append({"role": "assistant", "tool_calls": [tool_call], "content": ""})

    # Parse arguments and simulate function execution
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    result = ""
    if name == "get_weather":
        # Dummy weather response
        result = "15°C"
    elif name == "get_food_items":
        result = json.dumps(
            [
                {"id": 1, "name": "Baguettes", "quantity": 50},
                {"id": 2, "name": "Croissants", "quantity": 30},
            ]
        )

    # Append the tool result message
    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result})

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion.choices[0].message.content)

Today in Paris, the temperature is 15°C. As for the food items available, there are 50 baguettes and 30 croissants.
